# Implementing Self-Refine technique using Large Language Models

# Installing the dependencies

In [1]:
!pip install "mirascope[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.2/373.2 kB 5.3 MB/s eta 0:00:00


## OpenAI API Key
To get an OpenAI API key, visit https://platform.openai.com/settings/organization/api-keys and generate a new key. If you're a new user, you may need to add billing details and make a minimum payment of $5 to activate API access.

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass('Enter OpenAI API Key: ')

Enter OpenAI API Key: ··········


# Basic Self-Refine Implementation
This is a basic self-refine implementation using Mirascope and OpenAI’s gpt-4o-mini model. It starts by generating an initial answer to a math problem, then evaluates the response using structured feedback. That feedback is used to generate a revised answer, allowing iterative improvement. The refinement depth is configurable, making it suitable for tasks that benefit from multiple review cycles.

In [8]:
from mirascope.core import openai, prompt_template
from mirascope.core.openai import OpenAICallResponse


@openai.call(model="gpt-4o-mini")
def call(query: str) -> str:
    return query


@openai.call(model="gpt-4o-mini")
@prompt_template(
    """
    Here is a query and a response to the query. Give feedback about the answer,
    noting what was correct and incorrect.
    Query:
    {query}
    Response:
    {response}
    """
)
def evaluate_response(query: str, response: OpenAICallResponse): ...


@openai.call(model="gpt-4o-mini")
@prompt_template(
    """
    For this query:
    {query}
    The following response was given:
    {response}
    Here is some feedback about the response:
    {feedback}

    Consider the feedback to generate a new response to the query.
    """
)
def generate_new_response(
    query: str, response: OpenAICallResponse
) -> openai.OpenAIDynamicConfig:
    feedback = evaluate_response(query, response)
    return {"computed_fields": {"feedback": feedback}}


def self_refine(query: str, depth: int) -> str:
    response = call(query)
    for _ in range(depth):
        response = generate_new_response(query, response)
    return response.content


query = "A train travels 120 km at a certain speed. If the speed had been 20 km/h faster, it would have taken 30 minutes less to cover the same distance. What was the original speed of the train?"

print(self_refine(query, 1))

Let's start over with a clean and structured approach to the problem, ensuring clarity in each step while also emphasizing the importance of the conclusions drawn.

---

**Problem Statement:**

A train travels 120 km at a certain speed. If the speed had been 20 km/h faster, it would have taken 30 minutes less to cover the same distance. What was the original speed of the train?

**Solution:**

Let the original speed of the train be \( v \) km/h. 

1. **Calculate the time taken at the original speed:**
   \[
   \text{Time at original speed} = \frac{120}{v} \text{ hours}
   \]

2. **Calculate the time taken at the faster speed:**
   If the speed had been 20 km/h faster, the new speed is \( v + 20 \) km/h. Therefore, the time taken to cover 120 km at this speed is:
   \[
   \text{Time at faster speed} = \frac{120}{v + 20} \text{ hours}
   \]

3. **Establish the time difference:**
   According to the problem, the difference in time between traveling at the original speed and the faster spe

# Enhanced Self-Refine with Response Model
This is an enhanced self-refine implementation tailored for solving math problems using Mirascope and Pydantic. Instead of returning a raw string, it structures the output using a MathSolution model that clearly separates the step-by-step reasoning from the final numerical answer. After generating an initial response, the system evaluates its quality and uses structured feedback to produce a refined, model-conformant solution. This approach improves clarity, enables downstream validation, and supports more reliable multi-step reasoning in mathematical tasks.

In [9]:
from pydantic import BaseModel, Field


class MathSolution(BaseModel):
    steps: list[str] = Field(..., description="The steps taken to solve the problem")
    final_answer: float = Field(..., description="The final numerical answer")


@openai.call(model="gpt-4o-mini", response_model=MathSolution)
@prompt_template(
    """
    For this query:
    {query}
    The following response was given:
    {response}
    Here is some feedback about the response:
    {feedback}

    Consider the feedback to generate a new response to the query.
    Provide the solution steps and the final numerical answer.
    """
)
def enhanced_generate_new_response(
    query: str, response: OpenAICallResponse
) -> openai.OpenAIDynamicConfig:
    feedback = evaluate_response(query, response)
    return {"computed_fields": {"feedback": feedback}}


def enhanced_self_refine(query: str, depth: int) -> MathSolution:
    response = call(query)
    for _ in range(depth):
        solution = enhanced_generate_new_response(query, response)
        response = f"Steps: {solution.steps}\nFinal Answer: {solution.final_answer}"
    return solution


# Example usage
result = enhanced_self_refine(query, 1)
print(result)

steps=['Let the original speed of the train be v km/h.', 'The time taken to travel 120 km at this speed is Time = 120/v hours.', 'With a speed of (v + 20) km/h, the time taken is Time = 120/(v + 20) hours.', 'The difference in time taken is set as 30 minutes or 0.5 hours: 120/v - 120/(v + 20) = 0.5.', 'Clearing the fractions by multiplying through by 2v(v + 20): 2v(v + 20)(120/v - 120/(v + 20)) = 2v(v + 20)(0.5).', 'This simplifies to: 240(v + 20) - 240v = v(v + 20).', 'Resulting in: 240v + 4800 - 240v = v^2 + 20v, which simplifies to 4800 = v^2 + 20v.', 'Rearranging gives the quadratic equation: v^2 + 20v - 4800 = 0.', 'Applying the quadratic formula v = (-b ± √(b^2 - 4ac)) / 2a with a = 1, b = 20, c = -4800.', 'Calculating the discriminant: 20^2 - 4(1)(-4800) = 400 + 19200 = 19600.', 'Finding v values: v = (-20 ± √19600) / 2 = (-20 ± 140) / 2.', 'The valid solution: v = (120) / 2 = 60 km/h.'] final_answer=60.0
